### Find albums in Spotify and add songs to a playlist

In [1]:
import requests
import json
!pip install pandas
import pandas as pd
!pip install jsontable
import jsontable
import io
import time

     |████████████████████████████████| 9.9 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 39.3 MB/s eta 0:00:01


### Set Global Date Variable
Allows specification of which source CSV to pull and helps to ensure unique playlists are created

In [2]:
pullDate = "20210220"

### Token Management
Right now I have to manually refresh the token via https://developer.spotify.com/console

In [5]:
TOKEN = "BQDXlZfFMX6d3jjz-Sq4o9DON5IiQ-KpgIJJ5eMuiVhAqJA4hBj6QuYot6DjRWAuPaNix9icRZJUZXQxnwKJeQWgfOhPti66CCWlvS8ylROKVOogyx_dwSasKbDDgsnMMQYgwzXl_i1estCeWBvwvBoZyUOVAYBlja2gB9s8VNpX89JresqLQcmH3jGxzeQNdpsbCxM1cMmLocAD0pqVaq2lpuhznQJn"

### Header Construction

In [6]:
header = {"Authorization": "Bearer {}".format(TOKEN)}

### Request Construction
> Remember that Spotify will return a max of 20 objects by default, so artists with lots of albums might not truly be returning the most recent

In [7]:
artist = "bastille"
artist = artist.replace(" ", "%20")
artist = "artist:%22" + artist + "%22"
# album = "Bad Blood"
# album = album.replace(" ", "%20")
# album = "album:%22" + album + "%22"
# queryParam = album + artist
target = "https://api.spotify.com/v1/search?q={}&type=album".format(artist)
request = requests.get(target, headers = header).json()
print(json.dumps(request, indent=2))

{
  "albums": {
    "href": "https://api.spotify.com/v1/search?query=artist%3A%22bastille%22&type=album&offset=0&limit=20",
    "items": [
      {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/7EQ0qTo7fWT7DPxmxtSYEc"
            },
            "href": "https://api.spotify.com/v1/artists/7EQ0qTo7fWT7DPxmxtSYEc",
            "id": "7EQ0qTo7fWT7DPxmxtSYEc",
            "name": "Bastille",
            "type": "artist",
            "uri": "spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc"
          }
        ],
        "available_markets": [
          "US"
        ],
        "external_urls": {
          "spotify": "https://open.spotify.com/album/64fQ94AVziavTPdnkCS6Nj"
        },
        "href": "https://api.spotify.com/v1/albums/64fQ94AVziavTPdnkCS6Nj",
        "id": "64fQ94AVziavTPdnkCS6Nj",
        "images": [
          {
            "height": 640,
            "url": "https://i.scdn.co/image/a

### Function Accepts CSV as Input and Pulls New Albums

Testing out artist searches that resulted in errors

In [11]:
newArtist = "L'FREAQ"
newArtist = newArtist.replace(" ", "%20")
newArtist = newArtist.replace("&", "%26")
newArtist = newArtist.replace("'", "%27")
newArtist = newArtist.replace(".", "%2E")
newArtist = "artist:%22" + newArtist + "%22"
target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
print(target)
request = requests.get(target, headers = header).json()
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
print(len(df.index))
print(df)
# paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
# converter = jsontable.converter()
# converter.set_paths(paths)
# resultTable = converter.convert_json(request)
# df = pd.DataFrame(data=resultTable)
# df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
# df = df.drop(df.index[0])
# df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
# newest = pd.DataFrame(data=df.iloc[[0]])

https://api.spotify.com/v1/search?q=artist:%22L%27FREAQ%22&type=album
1
    0           1       2             3
0  ID  Album Name  Tracks  Release Date


In [8]:
# url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/New%20Music%20Source%20Files/iTunes%20Alternative%20{}.csv".format(pullDate)
url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/sampleArtists.csv"
fetch = requests.get(url).content
artists = pd.read_csv (io.StringIO(fetch.decode('utf-8')))
artists.style

newAlbums = pd.DataFrame()
errors = []

def pullAlbums(artistDF):
    
    global newAlbums
    global errors
    
    for index, row in artistDF.iterrows():
        # Wait five seconds to avoid 20 calls/minute limit on iTunes API
        # time.sleep(5)
        newArtist = row["Artist"]
        newArtist = newArtist.replace(" ", "%20")
        newArtist = newArtist.replace("&", "%26")
        newArtist = newArtist.replace("'", "%27")
        newArtist = newArtist.replace(".", "%2E")
        newArtist = "artist:%22" + newArtist + "%22"
        target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
        request = requests.get(target, headers = header).json()
        paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        resultTable = converter.convert_json(request)
        df = pd.DataFrame(data=resultTable)
        df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
        if len(df.index) > 1:
            df = df.drop(df.index[0])
            df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
            newest = pd.DataFrame(data=df.iloc[[0]])
            newAlbums = newAlbums.append(newest)
        else:
            errors.append(row["Artist"])
            continue

    print(newAlbums)
    # newAlbums.style
    # Would prefer to have the output styled in this manner but am getting "style is not supported for non-unique indices"
    print(errors)
    
pullAlbums(artists)

                 Album ID                       Album Name Tracks Release Date
3  2NKFtTrA1svqga0EUz2hQ1            Stores Will Foreclose      1   2019-11-22
7  6IE2H910UpaPbAEzS30usF                    Overabundance      3   2020-07-03
4  34toabo70RBibjJHvfkkbu   The Fight (Remixed / Extended)     12   2021-01-01
2  3hnKGmZQDeTlSoHzEq7sUX  how u stop hating urself (pt 1)      6   2020-12-04
2  0TsZ2qtOgqWP6wvLOzzlmM   Weeping Icon on Audiotree Live      5   2019-12-05
[]


### Pull Album ID Out of Response
Parse some key fields, and sort by both release date and number of tracks

In [5]:
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
df = df.drop(df.index[0])
df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
newest = pd.DataFrame(data=df.iloc[[0]])
newAlbums = pd.DataFrame()
newAlbums = newAlbums.append(newest)
newAlbums.style

,Album ID,Album Name,Tracks,Release Date
2,5G6oMu9zNW2acdV0lqzI3L,All This Bad Blood,25,2013-01-01


### Create Spotify Playlist
> Be careful here because Spotify will let you create playlists with the same name and description, assigning it a new playlist ID

In [9]:
userID = "t1e947vjw65dme71utp690f50"
createPlaylistTarget = "https://api.spotify.com/v1/users/{}/playlists".format(userID)
# payload = {"name": pullDate, "description": "Created from Jupyter notebook", "public": False}
payload = {"name": "Test Playlist", "description": "Created from Jupyter notebook", "public": False}
payload = json.dumps(payload, indent=2)
createPlaylistPOST = requests.post(createPlaylistTarget, headers = header, data = payload).json()
print(json.dumps(createPlaylistPOST, indent=2))

{
  "collaborative": false,
  "description": "Created from Jupyter notebook",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/6y0tFQ0hJvp63GowPd6COW"
  },
  "followers": {
    "href": null,
    "total": 0
  },
  "href": "https://api.spotify.com/v1/playlists/6y0tFQ0hJvp63GowPd6COW",
  "id": "6y0tFQ0hJvp63GowPd6COW",
  "images": [],
  "name": "Test Playlist",
  "owner": {
    "display_name": "Patrick",
    "external_urls": {
      "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
    },
    "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
    "id": "t1e947vjw65dme71utp690f50",
    "type": "user",
    "uri": "spotify:user:t1e947vjw65dme71utp690f50"
  },
  "primary_color": null,
  "public": false,
  "snapshot_id": "MSw4NzQwZmMxMjIyM2E2NzZmMGJiMzBhMzRlNGVjMmQwOTY3NGQzMTU4",
  "tracks": {
    "href": "https://api.spotify.com/v1/playlists/6y0tFQ0hJvp63GowPd6COW/tracks",
    "items": [],
    "limit": 100,
    "next": null,
    "off

Retrieve User Information

In [19]:
getUserTarget = "https://api.spotify.com/v1/users/{}".format(userID)
getUser = requests.get(getUserTarget, headers = header).json()
print(json.dumps(getUser, indent=2))

{
  "display_name": "Patrick",
  "external_urls": {
    "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
  },
  "followers": {
    "href": null,
    "total": 1
  },
  "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
  "id": "t1e947vjw65dme71utp690f50",
  "images": [
    {
      "height": null,
      "url": "https://i.scdn.co/image/ab6775700000ee85804e93a20b51ae91923b1a51",
      "width": null
    }
  ],
  "type": "user",
  "uri": "spotify:user:t1e947vjw65dme71utp690f50"
}


### Fetch Track IDs for Album
Store tracks in a dataframe and potentially sort on popularity (not seeing clear metadata to support this) 
Probably makes sense to add to the playlist in this loop so that it's adding chunks of tracks per album and any kind of error handling can be associated with the album.

In [28]:
playlistID = "6y0tFQ0hJvp63GowPd6COW"
newTracks = pd.DataFrame()
popVals = []

def assembleTracks(albumDF, playlist):
    
    global newTracks
    global popVals
    
    # Assemble request to add the album's tracks to the target playlist which is passed as function parameter
    addTracksTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist)
    
    for index, row in albumDF.iterrows():
        # Wait five seconds to avoid rate limit
        # time.sleep(5)
        newAlbumID = row["Album ID"]
        fetchTrackTarget = "https://api.spotify.com/v1/albums/{}/tracks".format(newAlbumID)
        fetchTrack = requests.get(fetchTrackTarget, headers = header).json()
        # parse JSON for appropriate track metadata to store
        paths = [{"$.items.id":"Track ID"}, {"$.items.name":"Track Name"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        trackTable = converter.convert_json(fetchTrack)
        df = pd.DataFrame(data=trackTable)
        df.columns = ["Track ID", "Track Name"]
        df = df.drop(df.index[0])
        newTracks = newTracks.append(df)

        # Assemble track URIs to add to payload for request to add tracks to playlist
        trackURIs = []
        
        for index, row in df.iterrows():
            selectedTrack = row["Track ID"]
            selectedTrackURI = "spotify:track:{}".format(selectedTrack)
            trackURIs.append(selectedTrackURI)
            print(trackURIs)
            
            # Retrieve popularity for each track
            
            getPopularityTarget = "https://api.spotify.com/v1/tracks/{}".format(selectedTrack)
            trackDetails = requests.get(getPopularityTarget, headers = header).json()
            
            # Parse popularity values and add to newTracks dataframe
            
            paths = [{"$.popularity":"Popularity"}]
            converter = jsontable.converter()
            converter.set_paths(paths)
            popularityColumn = converter.convert_json(trackDetails)
            tempPop = pd.DataFrame(data=popularityColumn)
            tempPop = tempPop.drop(tempPop.index[0])
            print(tempPop)
            newVal = tempPop.iloc[0][0]
            print(newVal)
            popVals.append(newVal)
            print(popVals)
        
        # Assemble payload
        payload = {"uris": trackURIs, "position":"null"}
        payload = json.dumps(payload, indent=2)
        
        # Execute request
        addTracks = requests.post(addTracksTarget, headers = header, data = payload).json()
    
    newTracks["Popularity"] = popVals    
    print(newTracks)
        
assembleTracks(newAlbums, playlistID)

['spotify:track:28kxBIEIi5SymMdtP9FaCz']
   0
1  1
1
[1]
['spotify:track:2nADPMfDOKyuYyH2MdsRkL']
   0
1  1
1
[1, 1]
['spotify:track:2nADPMfDOKyuYyH2MdsRkL', 'spotify:track:3g9SiuysgsvClKCOT94Szc']
   0
1  1
1
[1, 1, 1]
['spotify:track:2nADPMfDOKyuYyH2MdsRkL', 'spotify:track:3g9SiuysgsvClKCOT94Szc', 'spotify:track:6Z5X55jWYygw0UntugrISP']
   0
1  0
0
[1, 1, 1, 0]
['spotify:track:4LBWPdvJiRIEYhv1VxTaXQ']
    0
1  19
19
[1, 1, 1, 0, 19]
['spotify:track:4LBWPdvJiRIEYhv1VxTaXQ', 'spotify:track:657vAPXREGQwUwYg27oXlP']
    0
1  16
16
[1, 1, 1, 0, 19, 16]
['spotify:track:4LBWPdvJiRIEYhv1VxTaXQ', 'spotify:track:657vAPXREGQwUwYg27oXlP', 'spotify:track:6NfmWgGXkweCuvdHqbiMia']
    0
1  27
27
[1, 1, 1, 0, 19, 16, 27]
['spotify:track:4LBWPdvJiRIEYhv1VxTaXQ', 'spotify:track:657vAPXREGQwUwYg27oXlP', 'spotify:track:6NfmWgGXkweCuvdHqbiMia', 'spotify:track:7cVmyM2cwSxuQfbIVjh0pm']
    0
1  10
10
[1, 1, 1, 0, 19, 16, 27, 10]
['spotify:track:4LBWPdvJiRIEYhv1VxTaXQ', 'spotify:track:657vAPXREGQwUwYg27oXlP

In [29]:
pd.set_option("display.max_rows", None)
display(newTracks)

,Track ID,Track Name,Popularity
1,28kxBIEIi5SymMdtP9FaCz,Stores Will Foreclose,1
1,2nADPMfDOKyuYyH2MdsRkL,Front Row,1
2,3g9SiuysgsvClKCOT94Szc,Old Afternoon,1
3,6Z5X55jWYygw0UntugrISP,Slinky,0
1,4LBWPdvJiRIEYhv1VxTaXQ,I'll Be There - Shura Remix,19
2,657vAPXREGQwUwYg27oXlP,Apathetic Boys - Porches Remix,16
3,6NfmWgGXkweCuvdHqbiMia,New Shoes - Yoke Lore Remix,27
4,7cVmyM2cwSxuQfbIVjh0pm,Keep The Faith - Billy Lemos Remix,10
5,5deHXW5WHQMLSyxcrTON3i,Fire & Fury - Yumi Zouma Remix,11
6,52qRm2XREVJ5EspmbGSfkS,The Fool - GUM & Ginoli Remix,15


### Add Tracks to a Playlist
POST https://api.spotify.com/v1/playlists/{playlist_id}/tracks  
Need the playlist ID and a list of track URIs  

> Folded this into the assembleTracks function, so the logic below is now just a reference for making a single call

In [9]:
playlistID = "25NbFuQM1kDleGz1bjGJ2x"
addTrackTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlistID)
payload = {"uris": ["spotify:track:57As4lhkpp6tB422daefKK","spotify:track:3ePeWP6f8ccQtF7wayoNrD"], "position":"null"}
payload = json.dumps(payload, indent=2)
addTrack = requests.post(addTrackTarget, headers = header, data = payload).json()
print(json.dumps(addTrack, indent=2))

{
  "snapshot_id": "MTIsMzBmODE0NzUwOGVlNGJmZWZmZWYyMjExOGI2NzYyNjQzNzY5ZDExNA=="
}
